# CUDA Verification

This notebook verifies GPU access and CUDA functionality in your container environment.

**What you'll learn:**
- Check if CUDA is available
- View GPU properties
- Test basic tensor operations on GPU
- Benchmark CPU vs GPU performance

## 1. Import Required Libraries

In [1]:
import torch
import time
import platform

print(f"Python version: {platform.python_version()}")
print(f"PyTorch version: {torch.__version__}")

Python version: 3.11.6
PyTorch version: 2.5.1


## 2. Check CUDA Availability

In [2]:
cuda_available = torch.cuda.is_available()
print(f"CUDA Available: {cuda_available}")

if cuda_available:
    print(f"CUDA Version: {torch.version.cuda}")
    print(f"cuDNN Version: {torch.backends.cudnn.version()}")
    print(f"Number of GPUs: {torch.cuda.device_count()}")
else:
    print("\n⚠️  GPU not detected. Check that:")
    print("   1. Container was started with --device=nvidia.com/gpu=all")
    print("   2. Host system has ujust setup-gpu-containers configured")
    print("   3. NVIDIA drivers are properly installed on host")

CUDA Available: True
CUDA Version: 12.4
cuDNN Version: 90100
Number of GPUs: 1


## 3. GPU Device Properties

In [3]:
if cuda_available:
    for i in range(torch.cuda.device_count()):
        props = torch.cuda.get_device_properties(i)
        print(f"\nGPU {i}: {torch.cuda.get_device_name(i)}")
        print(f"  Compute Capability: {props.major}.{props.minor}")
        print(f"  Total Memory: {props.total_memory / 1024**3:.2f} GB")
        print(f"  Multiprocessors: {props.multi_processor_count}")
        print(f"  CUDA Cores (estimated): {props.multi_processor_count * 128}")


GPU 0: NVIDIA GeForce RTX 4080 SUPER
  Compute Capability: 8.9
  Total Memory: 15.57 GB
  Multiprocessors: 80
  CUDA Cores (estimated): 10240


## 4. Test Tensor Operations on GPU

In [4]:
if cuda_available:
    # Create tensors on GPU
    device = torch.device("cuda:0")
    
    # Create random tensors
    a = torch.randn(3, 3, device=device)
    b = torch.randn(3, 3, device=device)
    
    # Perform matrix multiplication
    c = torch.matmul(a, b)
    
    print("✓ Successfully created and multiplied tensors on GPU")
    print(f"\nResult tensor shape: {c.shape}")
    print(f"Result tensor device: {c.device}")
    print(f"\nSample output:\n{c}")

✓ Successfully created and multiplied tensors on GPU

Result tensor shape: torch.Size([3, 3])
Result tensor device: cuda:0

Sample output:
tensor([[ 1.1288,  1.6837,  2.8337],
        [ 1.1353,  2.3751,  1.8922],
        [ 0.0608,  1.1372, -0.1349]], device='cuda:0')


## 5. Performance Benchmark: CPU vs GPU

In [5]:
size = 5000
iterations = 10

# CPU benchmark
a_cpu = torch.randn(size, size)
b_cpu = torch.randn(size, size)

start = time.time()
for _ in range(iterations):
    c_cpu = torch.matmul(a_cpu, b_cpu)
cpu_time = time.time() - start

print(f"CPU Time ({iterations} iterations): {cpu_time:.4f} seconds")

# GPU benchmark (if available)
if cuda_available:
    device = torch.device("cuda:0")
    a_gpu = torch.randn(size, size, device=device)
    b_gpu = torch.randn(size, size, device=device)
    
    # Warm up GPU
    for _ in range(5):
        _ = torch.matmul(a_gpu, b_gpu)
    torch.cuda.synchronize()
    
    # Actual benchmark
    start = time.time()
    for _ in range(iterations):
        c_gpu = torch.matmul(a_gpu, b_gpu)
    torch.cuda.synchronize()
    gpu_time = time.time() - start
    
    print(f"GPU Time ({iterations} iterations): {gpu_time:.4f} seconds")
    print(f"\n🚀 GPU Speedup: {cpu_time/gpu_time:.2f}x faster")
else:
    print("\n⚠️  GPU not available for benchmark")

CPU Time (10 iterations): 2.7173 seconds
GPU Time (10 iterations): 0.0700 seconds

🚀 GPU Speedup: 38.82x faster


## Summary

If all cells executed successfully:
- ✅ CUDA is properly configured
- ✅ GPU is accessible from PyTorch
- ✅ GPU operations are working correctly
- ✅ GPU provides significant performance advantage

**Next steps:**
- Try the PyTorch Quickstart notebook (02-pytorch-quickstart.ipynb)
- Explore the Data Science Template (03-data-science-template.ipynb)
- Start your own ML/AI projects!